In [15]:
import numpy as np

In [16]:
##wb是模型本身的参数因此放在构造函数中无需手动进行更改 其他则由输入的x决定
class Affine:
    def __init__(self,w,b):
        self.w=w
        self.b=b
        self.x=None
        self.dw=None
        self.db=None
        self.dx=None
    def forward(self,x):
        if x.ndim==1:
            x = x.reshape(1,-1)
        self.x=x
        output=np.dot(self.x, self.w)+self.b
        return output
    def backward(self,d_out):
        self.dx=np.dot(d_out,self.w.T)
        self.dw=np.dot(self.x.T,d_out)
        self.db=np.sum(d_out,axis=0)

class Relu:
    def __init__(self):
        self.mask=None
    def forward(self,x):
        self.mask=x>0
        out=self.mask*x
        return out
    def backward(self,d_out):
        dx=self.mask*d_out
        return dx



class Softmaxwithloss:
    def __init__(self):
        self.t=None
        self.x=None
        self.y=None
        self.dx=None
        self.batch_size = None
    def forward(self,x):
        if x.ndim==1:
            x = x.reshape(1,-1)
        self.x=x-np.max(x,axis=-1,keepdims=True)
        self.batch_size = x.shape[0]
        self.y=np.exp(self.x)/np.sum(np.exp(self.x),axis=-1,keepdims=True)
        return self.y
    def loss(self,x,t):
        out=self.forward(x)
        self.t=t
        if t.ndim!=1:
            loss_rate=np.sum(-self.t*np.log(out+1e-7))/self.batch_size
        else:
            loss_rate=-np.sum(np.log(out[np.arange(len(t)),t]+1e-7))/self.batch_size
        return  loss_rate
    
    def backward(self):

        if self.t.ndim!=1:
            dx = (self.y - self.t) / self.batch_size
        else:
            y_c=self.y.copy()
            # dx=(y_c[np.arange(len(self.t)),self.t]-1)/self.batch_size 这里错了
            y_c[np.arange(len(self.t)),self.t] -=1
            dx=y_c/self.batch_size
        self.dx=dx
        return self.dx


In [ ]:
class MultiLayersNetwork:
    def __init__(self, input_size, output_size, hidden_size_list=None):
        if hidden_size_list is None:
            self.hidden_size_list = [100, 100, 100]
        self.input_size = input_size
        self.hidden_size_list = hidden_size_list
        self.output_size = output_size
        self.params=dict()
        self.sourcedata=None
        self.layers= dict()
        #生成层
        parameter_size_list=self.hidden_size_list.insert(0,self.input_size)
        parameter_size_list.append(self.output_size)
        
        ##倒数第二层之前全部用relu
        for i in range(len(parameter_size_list)-2):
            self.params['W'+str(i)]=np.random.rand(parameter_size_list[i],parameter_size_list[i+1])
            self.params['b'+str(i)]=np.random.rand(parameter_size_list[i],parameter_size_list[i+1])
            #每一层就自己层的权重和偏置
            self.layers['affine'+str(i)]=Affine(self.params['W'+str(i)],self.params['b'+str(i)])
            self.layers['Activation_function'+str(i)]=Relu()
        
        if i==len(parameter_size_list)-3:
            i+=1
            self.layers['affine'+str(i)]=Affine(self.params['W'+str(i)],self.params['b'+str(i)])
            self.layers['Activation_function'+str(i)]=Softmaxwithloss()
    

    def forward(self, x):
        inputs=x
        for key,func in self.layers.items:
            inputs=func.forward(inputs)

            